In [92]:
import pandas as pd
import numpy as np
import ast
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utilities import *
from music21.harmony import chordSymbolFigureFromChord as figureChord
from music21.chord import Chord

from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import model_from_json

import warnings

warnings.filterwarnings(action="once")

LSTM requirements:

- The LSTM input layer must be 3D.
- LSTMs don’t like sequences of more than 200-400 time steps, so the data will need to be split into samples.
- If you have a long sequence of thousands of observations, you must split it into samples and then reshape it for your LSTM model.
- The LSTM needs data with the format of [samples, time steps and features].
- The LSTM input layer is defined by the input_shape argument on the first hidden layer.
- The input_shape argument takes a tuple of two values that define the number of time steps and features.
- The number of samples is assumed to be 1 or more.
- The reshape() function on NumPy arrays can be used to reshape your 1D or 2D data to be 3D.
- The reshape() function takes a tuple as an argument that defines the new shape.


In [42]:
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
import utilities as my_utils
ft = FastText.load("./embeddings/fastText.model")
songs = my_utils.build_sentences()

In [ ]:
def prep_data(songs, sample_len = 4):
    # Remove too short songs
    songs = [chords for chords in songs if len(chords) > sample_len]
    x = []
    y = []
    for chords in songs:
        for i in range(len(chords)):
            if i < len(chords) - SAMPLE_LEN:
                x.append(chords[i: i + SAMPLE_LEN - 1])
            else:
                x.append(
                    chords[i: len(chords) - 1]
                    + chords[: (i + SAMPLE_LEN) % len(chords)]
                )
            y.append(chords[(i + SAMPLE_LEN) % len(chords)])
    x = np.array(x)
    y = np.array(y)
    print(x.shape, y.shape)
    return x, y

X, y = prep_data(songs)

In [87]:
def encode_chords(X, y, model):
    X_embedded = []
    y_embedded = []
    for X_sapmle, y_sapmle in zip(X, y):
        X_embedded.append(model.wv[X_sapmle])
        y_embedded.append(model.wv[y_sapmle])
    X_embedded = np.array(X_embedded)
    y_embedded = np.array(y_embedded)
    return X_embedded, y_embedded

(1243944, 3, 64) (1243944, 64) (1243944, 3) (1243944,)


In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, X_train, y_test = train_test_split(X_embedded, y_embedded, test_size=0.2, random_state=0)

In [130]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

In [131]:
print(y_embedded[0])

[-2.82 2.67 1.11 -1.31 -0.172 0.652 0.501 -1.35 -1.25 0.533 0.434 -0.0232 1.74 -0.362 -2.12 1.63 -1.46 0.598 -0.459 0.809 0.9 3.27 0.758 -2.99 0.793 -0.149 -0.986 -0.328 1.69 -0.752 -0.222 0.957 1.51 -1.28 0.333 1.04 -1.94 -1.66 -0.0551 -0.472 0.308 -0.306 -0.376 0.757 -1.05 -1.13 1.27 -2.23 2.5 -2.14 0.851 0.242 1.61 1.61 -1.38 1.08 0.17 -1.02 -0.448 0.032 1.2 2.13 0.238 -2.02]


# Test Model

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense, Dropout

model = keras.Sequential(
    [
        LSTM(64, input_shape=X_train.shape[1:]),
        Dense(y_train.shape[1], activation='sigmoid')
    ]
)

In [105]:
class Metrics(keras.callbacks.Callback):
    def __init__(embedding):
        self.embedding = embedding
        
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'val_rocauc': roc_auc_score(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data


In [137]:
validation_split = 0.25
epochs = 20
batch_size = 500
METRICS = [
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.BinaryAccuracy(name="accuracy"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(name="auc"),
]
METRICS_NAMES = [
    "loss",
    "tp",
    "fp",
    "tn",
    "fn",
    "accuracy",
    "precision",
    "recall",
    "auc",
]
loss = keras.losses.MeanSquaredError() 
optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(loss=loss, batch_size=batch_size, optimizer=optimizer, metrics=METRICS)
print(model.summary())
history = model.fit(X_train[:100000], y_train[:100000], epochs=3, batch_size=1000, validation_split=0.2).history

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
Total params: 37,184
Trainable params: 37,184
Non-trainable params: 0
_________________________________________________________________
None
Train on 80000 samples, validate on 20000 samples
Epoch 1/3
80000/80000 [==============================] - 13s 166us/sample - loss: 1.8671 - tp: 876017.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 4243983.0000 - accuracy: 0.0000e+00 - precision: 1.0000 - recall: 0.1711 - auc: 0.0000e+00 - val_loss: 1.8667 - val_tp: 224670.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 1055330.0000 - val_accuracy: 0.0000e+00 - val_precision: 1.0000 - val_recall: 0.1755 - val_auc: 0.0000e+00
Epoch 2/3
80000/800

In [138]:
history

{'loss': [1.8670565992593766, 1.8553203254938126, 1.8459146738052368],
 'tp': [876017.0, 910904.0, 956981.0],
 'fp': [0.0, 0.0, 0.0],
 'tn': [0.0, 0.0, 0.0],
 'fn': [4243983.0, 4209096.0, 4163019.0],
 'accuracy': [0.0, 0.0, 0.0],
 'precision': [1.0, 1.0, 1.0],
 'recall': [0.17109707, 0.17791094, 0.18691035],
 'auc': [0.0, 0.0, 0.0],
 'val_loss': [1.866728091239929, 1.857328337430954, 1.8482354521751403],
 'val_tp': [224670.0, 230723.0, 248049.0],
 'val_fp': [0.0, 0.0, 0.0],
 'val_tn': [0.0, 0.0, 0.0],
 'val_fn': [1055330.0, 1049277.0, 1031951.0],
 'val_accuracy': [0.0, 0.0, 0.0],
 'val_precision': [1.0, 1.0, 1.0],
 'val_recall': [0.17552343, 0.18025234, 0.19378828],
 'val_auc': [0.0, 0.0, 0.0]}

In [ ]:
X_test.shape

In [103]:
scores = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
Total params: 37,184
Trainable params: 37,184
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 0.00%


In [10]:
for i in range(20):
    predict_chords(loaded_model, X_test[i])

A-maj7/C  | A#m7/C#  | A-maj7/C  | Fm7/C
Fm/C  | Fm/C  | C7  | Fm/CaddB-
Gm/D  | B-7/D  | Fm/C  | Gm7/D
E-  | A-7/C  | A-7/C  | A-/CaddA
Gm7/D  | C7  | F/C  | F/CaddB-
E-  | Gm/D  | Cm  | Cm7
C7  | F-+/CaddB-  | F/C  | B-/C
B-/D  | E-+  | C7  | Chord Symbol Cannot Be Identified
F7/C  | B-/D  | B-/D  | B-/C
E-m  | A#m7/C#  | E-m  | Chord Symbol Cannot Be Identified
Gm7/D  | C7  | F/C  | B-/C
D/o7/C  | E-+addF  | Cm7  | B-/C
F9/CaddB  | B-7/D  | Cm7addD-,A-,F  | D-maj7/C
A7/C#  | D7/C  | D7/C  | Chord Symbol Cannot Be Identified
Fm7/C  | B-7/D  | E-maj7/D  | D/o7/C
Am7/C  | E7/D  | E7/D  | CaddD
B-7/D  | E-maj7/D  | B-/D  | Cm7
B-/D  | A7/C#  | B-/D  | E-addF
Gm7/D  | Gm7/D  | Cm7  | B-/C
A7/C#  | Dm  | G7/D  | Chord Symbol Cannot Be Identified
